In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
df_notes = pd.read_csv('../data/NOTEEVENTS.csv')

/home/mcb/users/pnair6/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_adm = pd.read_csv('../data/ADMISSIONS.csv')
df_icu = pd.read_csv('../data/ICUSTAYS.csv')
df_mechvent_d2 = pd.read_csv('../data/d2_mechvent_cohort08Oct19.csv')
df_mechvent_d7 = pd.read_csv('../data/d7_mechvent_cohort27Sep19.csv')
df_mechvent_d14 = pd.read_csv('../data/d14_mechvent_cohort27Sep19.csv')
df_mechvent_entire = pd.read_csv('../data/entire_mechvent_cohort_starttimes15Oct19.csv')

# add the INTIME Column from ICUSTAYS table
df_mechvent_d2 = pd.merge(df_mechvent_d2, df_icu[['ICUSTAY_ID', 'INTIME']], on = ['ICUSTAY_ID'], how = 'inner')
df_mechvent_entire = pd.merge(df_mechvent_entire, df_icu[['ICUSTAY_ID', 'INTIME']], on = ['ICUSTAY_ID'], how = 'inner')

In [4]:
df_mechvent_d2.head()

,ICUSTAY_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,ADMISSION_TYPE,ETHNICITY,HOSPITAL_EXPIRE_FLAG,GENDER,DOD,...,ICUSTAY_SEQ,FIRST_ICU_STAY,VENTNUM,STARTTIME,ENDTIME,DURATION_HOURS,DAYS_UNTIL_DEATH,FIRST_VENT_STARTTIME,COHORT,INTIME_y
0,211552,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,EMERGENCY,WHITE,0,M,2102-06-14,...,1,t,1,2101-10-20 20:00:00,2101-10-24 07:44:00,83.733333,236.202778,2101-10-20 20:00:00,d2,2101-10-20 19:10:11
1,220597,9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,EMERGENCY,UNKNOWN/NOT SPECIFIED,1,M,2149-11-14,...,1,t,1,2149-11-09 13:40:00,2149-11-14 12:00:00,118.333333,4.454167,2149-11-09 13:40:00,d2,2149-11-09 13:07:02
2,232669,12,112213,2104-08-07 10:15:00,2104-08-20 02:57:00,ELECTIVE,WHITE,1,M,2104-08-20,...,1,t,1,2104-08-08 04:20:00,2104-08-11 15:00:00,82.666667,12.572917,2104-08-08 04:20:00,d2,2104-08-08 02:08:17
3,254478,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,EMERGENCY,WHITE,1,M,2108-08-30,...,1,t,1,2108-08-23 15:00:00,2108-08-30 13:00:00,166.000000,7.022917,2108-08-23 15:00:00,d2,2108-08-22 23:28:42
4,282039,35,166707,2122-02-10 11:15:00,2122-02-20 15:30:00,ELECTIVE,WHITE,0,F,NaN,...,1,t,1,2122-02-10 13:00:00,2122-02-10 20:40:00,7.666667,NaN,2122-02-10 13:00:00,d2,2122-02-10 09:39:59


In [5]:
def process(df):
    '''
    Input
        df: the initial cohort
        
    output
        df_less_n_r: first icu stay cohort with notes within 48 hour starting from first vent time, with the following labels:
            COHORT:
                0 for not prolonged, 1 for more than 7 days, 2 for more than 14 days. So for prolonged for more than 7 days, should sum 1 & 2 cohort
            LABEL:
                0 for not prolonged, 1 for more than 7 days (including more than 14 days)
            DEATH_90:
                0 for not dead, 1 for dead within 90 days of 48 hours (action time) after first vent time 
            DAYS_UNTIL_DEATH: 
                continuous variable of days of death from the 
            
            this table only returns notes that are in the respiratory, nurses and physician category
       
    '''
    
    # only use the first ICU stays STARTING from the FIRST VENT TIME
    df = df.sort_values(['HADM_ID','INTIME_x']).groupby('HADM_ID', as_index=False).first()
    
    # drop all the repetitive ventilation events, so now this df has each row correspond to unique one admission's first icu stay
    df = df[['ICUSTAY_ID', 'HADM_ID', 'ADMITTIME','DISCHTIME', 'STARTTIME', 'ENDTIME', 'DURATION_HOURS', 'FIRST_VENT_STARTTIME', 'DOD']].drop_duplicates().reset_index(drop = True)
    
    df.DOD = pd.to_datetime(df.DOD, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df.ADMITTIME = pd.to_datetime(df.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df.DISCHTIME = pd.to_datetime(df.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df.FIRST_VENT_STARTTIME = pd.to_datetime(df.FIRST_VENT_STARTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df.ENDTIME = pd.to_datetime(df.ENDTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    
    # cohort 0: not prolonged, 1: more than 7 days, 2: more than 14 days
    df = df.assign(COHORT = [1 if i else 0 for i in (df.HADM_ID.isin(df_mechvent_d7.HADM_ID.unique()))])
    df.loc[df.HADM_ID.isin(df_mechvent_d14.HADM_ID.unique()), 'COHORT'] = 2
    count = df.groupby('COHORT').HADM_ID.nunique().values.tolist()
    print('Not prolonged: {}, more than 7 days: {}, more than 14 days: {}'.format(count[0], count[1], count[2]))
    
    #to calculate the death label
    df = df.assign(DAYS_UNTIL_DEATH = ((df['DOD']-df['FIRST_VENT_STARTTIME']).dt.total_seconds()/(60*60*24)))
    df = df.assign(DEATH = [1 if i else 0 for i in (df.DAYS_UNTIL_DEATH < 7)])
    df = df.assign(DEATH_90 = [1 if i else 0 for i in (df.DAYS_UNTIL_DEATH < 92)])
    
    df = df.assign(MV_DURATION = (df['ENDTIME'] - df['FIRST_VENT_STARTTIME']).dt.total_seconds()/(60*60))
    
    df_curated = df.drop_duplicates().reset_index(drop = True)
    print('n (number of admissions) is : {}'.format(len(df_curated.HADM_ID.unique())))
    
    # filter so that notes only in the admission cohort, in the CATEGORY we want
    df_notes_cohort = df_notes[df_notes.HADM_ID.isin(df_curated.HADM_ID)]
    print('removed subjects with no notes associated, n: {}'.format(len(df_notes_cohort.HADM_ID.unique())))

    df_notes_cohort = pd.merge(df_notes_cohort[['HADM_ID', 'CHARTTIME', 'TEXT', 'CATEGORY']], df_curated, on = ['HADM_ID'], how = 'inner')
    print('after merge, n: {}'.format(len(df_notes_cohort.HADM_ID.unique())))
    
    df_notes_cohort.CHARTTIME = pd.to_datetime(df_notes_cohort.CHARTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
    df_notes_cohort.ADMITTIME = pd.to_datetime(df_notes_cohort.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
        
    TIME_to_VENT = ((df_notes_cohort['CHARTTIME']-df_notes_cohort['FIRST_VENT_STARTTIME']).dt.total_seconds()/(60*60))
    df_less_n = df_notes_cohort[ (TIME_to_VENT<=48) & (TIME_to_VENT >=0)]
    print('restricted to the first 48h, n: {}'.format(len(df_less_n.HADM_ID.unique())))
    df_less_n = df_less_n.assign(Label = [1 if i else 0 for i in (df_less_n.COHORT != 0)])

    df_less_n_r = df_less_n[df_less_n.CATEGORY.isin(['Nursing', 'Physician ', 'Respiratory '])]

    print('remove subjects with no notes in the selected categories the entire admission, n: {}'.format(len(df_notes_cohort[df_notes_cohort.CATEGORY.isin(['Physician ', 'Nursing', 'Nursing/other'])].HADM_ID.unique())))
    print('remove subjects with no notes in the selected categories under 48h, n: {}'.format(len(df_less_n_r.HADM_ID.unique())))
    
    print('# of notes restricted to the three categories: {}'.format(len(df_less_n_r)))
    print('# of notes using all categories: {}'.format(len(df_less_n)))
    
    print('--- for prediction: below are label stats using three categories only ---')
    
    count = df_less_n_r.groupby('COHORT').HADM_ID.nunique().values.tolist()
    print('Not prolonged: {}, prolonged: {}, out of which, more than 7 days: {}, more than 14 days: {}'.format(count[0], count[1]+count[2], count[1], count[2]))
    count = df_less_n_r.groupby('DEATH').HADM_ID.nunique().values.tolist()
    print('Not Death within 7 days: {}, Death within 7 days: {}'.format(count[0], count[1]))
    
    df_death = df_less_n_r[df_less_n_r.HADM_ID.isin(df_less_n_r.groupby('DEATH').HADM_ID.unique()[1])]
    c = df_death[['HADM_ID','COHORT']].drop_duplicates().COHORT.value_counts().values
    try:
        print('Out of the {} death within 7 days, {} is from cohort not prolonged, {} is from cohort prolonged more than 7 days, and {} for 14 days'.format(count[1], c[0], c[1], c[2]))
    except:
        print('Out of the {} death within 7 days, {} is from cohort not prolonged, {} is from cohort prolonged more than 7 days, and 0 for 14 days'.format(count[1], c[0], c[1]))
        
    count = df_less_n_r.groupby('DEATH_90').HADM_ID.nunique().values.tolist()
    print('Not Death within 90 days: {}, Death within 90 days: {}'.format(count[0], count[1]))
      
    return df_less_n_r.reset_index(drop = True)
    #return df_less_n_r.reset_index(drop = True), df_less_n.reset_index(drop = True), df_physician.reset_index(drop = True), df_notes_cohort.reset_index(drop = True)
    

In [6]:
df_less_n_d2 = process(df_mechvent_d2)
#df_less_n_entire = process(df_mechvent_entire)

Not prolonged: 5185, more than 7 days: 2424, more than 14 days: 1874
n (number of admissions) is : 9483
removed subjects with no notes associated, n: 9372
after merge, n: 9372
restricted to the first 48h, n: 9095
remove subjects with no notes in the selected categories the entire admission, n: 7290
remove subjects with no notes in the selected categories under 48h, n: 1545
# of notes restricted to the three categories: 38246
# of notes using all categories: 120108
--- for prediction: below are label stats using three categories only ---
Not prolonged: 908, prolonged: 637, out of which, more than 7 days: 376, more than 14 days: 261
Not Death within 7 days: 1351, Death within 7 days: 194
Out of the 194 death within 7 days, 192 is from cohort not prolonged, 2 is from cohort prolonged more than 7 days, and 0 for 14 days
Not Death within 90 days: 1024, Death within 90 days: 521


In [7]:
df_less_n_d2.head()

,HADM_ID,CHARTTIME,TEXT,CATEGORY,ICUSTAY_ID,ADMITTIME,DISCHTIME,STARTTIME,ENDTIME,DURATION_HOURS,FIRST_VENT_STARTTIME,DOD,COHORT,DAYS_UNTIL_DEATH,DEATH,DEATH_90,MV_DURATION,Label
0,180836.0,2152-02-17 21:38:00,Clinician: Attending\n Evaluated pt who was...,Physician,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,5.0,0
1,180836.0,2152-02-17 15:04:00,Assessment/Plan: Mr. [**Known lastname **] is...,Nursing,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,5.0,0
2,180836.0,2152-02-17 15:04:00,Assessment/Plan: Mr. [**Known lastname **] is...,Nursing,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,5.0,0
3,180836.0,2152-02-17 15:04:00,Assessment/Plan: Mr. [**Known lastname **] is...,Nursing,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,5.0,0
4,180836.0,2152-02-17 18:31:00,Lung sounds\n RLL Lung Sounds: Diminished\n ...,Respiratory,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,5.0,0


In [8]:
df_less_n_d2 = df_less_n_d2.drop(["MV_DURATION"],axis=1)
df_less_n_d2.head()

,HADM_ID,CHARTTIME,TEXT,CATEGORY,ICUSTAY_ID,ADMITTIME,DISCHTIME,STARTTIME,ENDTIME,DURATION_HOURS,FIRST_VENT_STARTTIME,DOD,COHORT,DAYS_UNTIL_DEATH,DEATH,DEATH_90,Label
0,180836.0,2152-02-17 21:38:00,Clinician: Attending\n Evaluated pt who was...,Physician,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,0
1,180836.0,2152-02-17 15:04:00,Assessment/Plan: Mr. [**Known lastname **] is...,Nursing,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,0
2,180836.0,2152-02-17 15:04:00,Assessment/Plan: Mr. [**Known lastname **] is...,Nursing,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,0
3,180836.0,2152-02-17 15:04:00,Assessment/Plan: Mr. [**Known lastname **] is...,Nursing,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,0
4,180836.0,2152-02-17 18:31:00,Lung sounds\n RLL Lung Sounds: Diminished\n ...,Respiratory,243173,2152-02-15 22:33:00,2152-03-02 14:00:00,2152-02-17 09:00:00,2152-02-17 14:00:00,5.0,2152-02-17 09:00:00,2154-04-24,0,796.625,0,0,0


In [57]:
a = df_less_n_d2[df_less_n_d2.MV_DURATION.astype(float) < 168.0].Label.values
ind = [i for i,x in enumerate(a) if x == 1]
len(ind)

6785

In [9]:
import re
import string

def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('--|__|==','',y)
    
    # remove punctuation, digits, spaces
    #y = y.translate(str.maketrans("", "", string.punctuation))
    y = y.translate(str.maketrans("", "", string.digits))
    y = " ".join(y.split())
    return y

def preprocessing_note(df_less_n): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))
    
    df_less_n['# of tokens'] = df_less_n['TEXT'].str.split().str.len()
    df_less_n = df_less_n[df_less_n['# of tokens'] <=5]
    return df_less_n

In [10]:
df_use_d2 = preprocessing_note(df_less_n_d2) # day 2 cohort for training and cross-validation of classifier
#df_use_entire = preprocessing_note(df_less_n_entire) # entire cohort for learning topics using mixEHR

In [9]:
len(df_less_n_d2.HADM_ID.unique())

1545

In [12]:
# create one untouchable test set 
df_HADM_ID = df_less_n_d2.HADM_ID.drop_duplicates().reset_index(drop = True)
test_id = df_HADM_ID.sample(frac = 0.1, replace = False, random_state = 1)
test = df_less_n_d2[df_less_n_d2.HADM_ID.isin(test_id.values)]
test.reset_index(drop = True).to_csv('../data/data_files_2Dec/test.csv')

train_val_id = df_HADM_ID[~df_HADM_ID.index.isin(test_id.index)]

In [13]:
train_val_id = np.array(train_val_id.values)

In [14]:
five_folds = np.array_split(train_val_id,5)

In [15]:
for i, fold in enumerate(five_folds):
    f = df_less_n_d2[df_less_n_d2.HADM_ID.isin(fold)]
    print(f.shape[0])
    path = "../data/data_files_2Dec/fold" + str(i+1) +  ".csv"
    f.reset_index(drop = True).to_csv(path,index=False)

7207
7038
6760
6518
6975


In [29]:
len(train_val_id)

40

In [17]:
!ls ../data/data_files_2Dec

fold1.csv  fold2.csv  fold3.csv  fold4.csv  fold5.csv  test.csv


/home/mcb/users/pnair6/mimic/preprocessing
